In [1]:
import numpy as np
from numpy.typing import NDArray
from typing import Callable, Dict, Tuple

class KF:
    """
    Square-root Kalman filter in the spirit of Ramos (2021), using:

      - Square-root *time update* via QR:
            P_{k+1|k} = F_k P_{k|k} F_k^T + Q_k
        with P represented as S S^T and updated via a stacked QR.

      - Square-root *measurement update*:
        Joseph-form covariance update followed by a Cholesky factorization.

    The filter state is (x_hat, S), where S is a lower-triangular factor
    such that P = S S^T.
    """

    @staticmethod
    def f(
        *,
        xhat_P: Tuple[NDArray, NDArray],  # (x̂_k, S_k)
        yk: NDArray,
        f: Callable,
        f_params: Dict,
        F: Callable,
        F_params: Dict,
        Q: Callable,
        Q_params: Dict,
        h: Callable,
        h_params: Dict,
        H: Callable,
        H_params: Dict,
        R: Callable,
        R_params: Dict,
    ) -> Tuple[NDArray, NDArray]:
        """
        One full EKF step (predict + update) with square-root covariance.

        xhat_P = (x̂_k, S_k), where S_k is a Cholesky factor of P_k.
        """

        xk, Sk = xhat_P
        xk = np.asarray(xk, dtype=float).reshape(-1, 1)
        Sk = np.asarray(Sk, dtype=float)

        n = xk.shape[0]
        eps = 1e-9

        # --------------------- Time update ---------------------
        # State prediction
        x_pred = np.asarray(f(**f_params), dtype=float).reshape(n, 1)

        Fk = np.asarray(F(**F_params), dtype=float)
        Qk = np.asarray(Q(**Q_params), dtype=float)

        # Cholesky of Qk (with small ridge if needed)
        try:
            SQ = np.linalg.cholesky(Qk)
        except np.linalg.LinAlgError:
            SQ = np.linalg.cholesky(Qk + eps * np.eye(Qk.shape[0]))

        # Stack S_k F_k^T and SQ, then QR to get S_{k+1|k}
        A = np.vstack((Sk @ Fk.T, SQ))   # shape (2n, n)
        _, R_mat = np.linalg.qr(A, mode="reduced")  # R: (n, n), upper-triangular

        S_pred = R_mat.T  # lower-triangular square root of P_pred

        # --------------------- Measurement update ---------------------
        yk = np.asarray(yk, dtype=float).reshape(-1, 1)
        y_pred = np.asarray(h(**h_params), dtype=float).reshape(-1, 1)
        innovation = yk - y_pred   # (m,1)

        Hk = np.asarray(H(**H_params), dtype=float)
        Rk = np.asarray(R(**R_params), dtype=float)

        # Innovation covariance S_y = H P_pred H^T + R,
        # but compute P_pred from S_pred, then factor.
        P_pred = S_pred @ S_pred.T

        S_y = Hk @ P_pred @ Hk.T + Rk
        m = S_y.shape[0]

        try:
            S_Sy = np.linalg.cholesky(S_y + eps * np.eye(m))
        except np.linalg.LinAlgError:
            S_Sy = np.linalg.cholesky(S_y + 10 * eps * np.eye(m))

        # K = P_pred H^T S_y^{-1}
        # Use Cholesky solves for numerical stability.
        B = Hk @ P_pred         # (m, n)
        # Solve S_Sy^T X = B   ⇒  X = S_Sy^{-T} B = S_y^{-T} B
        X = np.linalg.solve(S_Sy.T, B)   # (m, n)
        Kk = X.T                 # (n, m) = P_pred H^T S_y^{-T}

        # State update
        x_upd = x_pred + Kk @ innovation

        # Covariance update (Joseph form)
        I = np.eye(n)
        P_upd = (I - Kk @ Hk) @ P_pred @ (I - Kk @ Hk).T + Kk @ Rk @ Kk.T
        # Symmetrize to kill round-off asymmetry
        P_upd = 0.5 * (P_upd + P_upd.T)

        # Re-factor P_upd as S_upd S_upd^T
        try:
            S_upd = np.linalg.cholesky(P_upd + eps * np.eye(n))
        except np.linalg.LinAlgError:
            # Fallback with slightly larger ridge if ill-conditioned
            S_upd = np.linalg.cholesky(P_upd + 10 * eps * np.eye(n))

        return (x_upd, S_upd)

    @staticmethod
    def h(xhat_P: Tuple[NDArray, NDArray]) -> NDArray:
        """
        Extract the current state estimate x̂_k from (x̂_k, S_k).
        """
        return np.asarray(xhat_P[0], dtype=float)


In [3]:
import pykal
kf = pykal.DynamicalSystem(f=KF.f, h=KF.h, state_name="xhat_P")


In [4]:
import numpy as np
# simulate dynamics 
dt = 0.1
pos_vel = np.asarray([0,2], dtype=float).reshape(2, 1)

pos_vel_hist = [pos_vel]
pos_hist = [pos_vel[0]]

P0 = Q_oscillator(pos_vel)  # or your chosen initial covariance
S0 = np.linalg.cholesky(P0)
xhat_P = [pos_vel, S0]

NameError: name 'Q_oscillator' is not defined